# Texas Rangers (TEX, WS2)

Years excluded due to shorted seasons (1981, 1994, 1995, 2020)

In [1]:
# Imports
import numpy as np
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

In [2]:
df1 = pd.read_csv('/kaggle/input/batting/Batting.csv')
df2 = pd.read_csv('/kaggle/input/homegames/HomeGames.csv')
df3 = pd.read_csv('/kaggle/input/parks/Parks.csv')
df4 = pd.read_csv('/kaggle/input/people/People.csv')
df5 = pd.read_csv('/kaggle/input/salaries/Salaries.csv')
df6 = pd.read_csv('/kaggle/input/teams/Teams.csv')

In [3]:
conn = sqlite3.connect(':memory')

In [4]:
df1.to_sql('batting', conn, index=False, if_exists='replace')
df2.to_sql('homegames', conn, index=False, if_exists='replace')
df3.to_sql('parks', conn, index=False, if_exists='replace')
df4.to_sql('people', conn, index=False, if_exists='replace')
df5.to_sql('salaries', conn, index=False, if_exists='replace')
df6.to_sql('teams', conn, index=False, if_exists='replace')

3045

1. Finding average number of wins (W) by my team between 2000-2023. 

In [5]:
query1 = '''
SELECT AVG(W) AS avg_wins
FROM teams
WHERE teamID IN ('TEX', 'WS2')
    AND yearID BETWEEN 2000 AND 2023
    AND yearID != 2020;
'''
print(pd.read_sql_query(query1, conn))

    avg_wins
0  79.869565


2. Sucess of the team between 2000-2023. Sucess includes winning the Wild Card (WCWin), Division (DivWin), League (LgWin), and World Series (WSWin)

In [6]:
query2 = '''
SELECT SUM(CASE WHEN WCWin = 'Y' THEN 1 ELSE 0 END) AS wild_card_wins,
    SUM(CASE WHEN DivWin = 'Y' THEN 1 ELSE 0 END) AS division_wins,
    SUM(CASE WHEN LgWin = 'Y' THEN 1 ELSE 0 END) AS league_wins,
    SUM(CASE WHEN WSWin = 'Y' THEN 1 ELSE 0 END) AS world_series_wins
FROM teams
WHERE teamID IN ('TEX', 'WS2')
    AND yearID BETWEEN 2000 AND 2023
    AND yearID != 2020;
'''
print(pd.read_sql_query(query2, conn))

   wild_card_wins  division_wins  league_wins  world_series_wins
0               2              4            3                  1


3. Ranking the team between 1st and 5th place in between 2000-2023 using the Rank field. 

In [7]:
query3 = '''
SELECT Rank, COUNT(*) AS times_ranked
FROM teams
WHERE teamID IN ('TEX', 'WS2')
    AND yearID BETWEEN 2000 AND 2023
    AND yearID != 2020
    AND Rank BETWEEN 1 AND 5
GROUP BY Rank
ORDER BY Rank;
'''
print(pd.read_sql_query(query3, conn))

   Rank  times_ranked
0     1             4
1     2             5
2     3             4
3     4             7
4     5             3


4. Maximum and minimum player salary records for the team between 1985 and 2016.

In [8]:
query4 = '''
SELECT yearID, MAX(salary), MIN(salary)
FROM salaries
WHERE teamID IN ('TEX', 'WS2')
    AND yearID BETWEEN 1985 AND 2016
    AND yearID != 1994
    AND yearID != 1995
GROUP BY yearID
ORDER BY yearID;
'''
print(pd.read_sql_query(query4, conn))

    yearID  MAX(salary)  MIN(salary)
0     1985       720000        60000
1     1986       865000        60000
2     1987       620000        62500
3     1988       850000        62500
4     1989      1800000        68000
5     1990      1633333       100000
6     1991      3300000       105000
7     1992      5000000       109000
8     1993      4800000       109000
9     1996      7425000       109000
10    1997      7500000       150000
11    1998      7800000       170000
12    1999      8950000       200000
13    2000      8620921       200000
14    2001     22000000       202500
15    2002     22000000       200000
16    2003     22000000       300000
17    2004     14000000       300000
18    2005     15000000       316000
19    2006     10472409       327000
20    2007      9836116       381000
21    2008     11000000       390000
22    2009     13054526       400000
23    2010     13174974       401000
24    2011     16174974       414000
25    2012     16174974       480000
2

 5. Creating a new field to describe player's salary group  (Key Player)

In [9]:
query5 = '''
SELECT playerID, salary,
CASE WHEN salary = (SELECT MAX(salary) FROM salaries) THEN 'Max'
    WHEN salary = (SELECT MIN(salary) FROM salaries) THEN 'Min'
    ELSE 'Neither'
    END AS key_player
FROM salaries;
'''
print(pd.read_sql_query(query5, conn))

        playerID    salary key_player
0      barkele01    870000    Neither
1      bedrost01    550000    Neither
2      benedbr01    545000    Neither
3       campri01    633333    Neither
4      ceronri01    625000    Neither
...          ...       ...        ...
26423  strasst01  10400000    Neither
26424  taylomi02    524000    Neither
26425  treinbl01    524900    Neither
26426  werthja01  21733615    Neither
26427  zimmery01  14000000    Neither

[26428 rows x 3 columns]


6. Joining the people spreadsheet to include players' names. 

In [10]:
query6 = '''
SELECT salaries.playerID, salaries.salary, people.nameFirst, people.nameLast
FROM salaries JOIN people USING(playerID);
'''
print(pd.read_sql_query(query6, conn))

        playerID    salary nameFirst   nameLast
0      barkele01    870000       Len     Barker
1      bedrost01    550000     Steve  Bedrosian
2      benedbr01    545000     Bruce   Benedict
3       campri01    633333      Rick       Camp
4      ceronri01    625000      Rick     Cerone
...          ...       ...       ...        ...
26423  strasst01  10400000   Stephen  Strasburg
26424  taylomi02    524000   Michael     Taylor
26425  treinbl01    524900     Blake    Treinen
26426  werthja01  21733615    Jayson      Werth
26427  zimmery01  14000000      Ryan  Zimmerman

[26428 rows x 4 columns]


7. Players with the max and min salary each year from 1985 - 2016

In [11]:
query7 = '''
WITH min_max_salaries AS (
    SELECT yearID,MAX(salary) AS max_salary,MIN(salary) AS min_salary
    FROM salaries
    WHERE yearID BETWEEN 1985 AND 2016
      AND yearID NOT IN (1994, 1995)
    GROUP BY yearID
)
SELECT s.yearID, s.playerID, p.nameFirst, p.nameLast, s.salary,
  CASE 
    WHEN s.salary = mm.max_salary THEN 'Max'
    WHEN s.salary = mm.min_salary THEN 'Min'
  END AS key_player
FROM salaries s JOIN people p USING (playerID) JOIN min_max_salaries mm ON s.yearID = mm.yearID
WHERE s.yearID BETWEEN 1985 AND 2016
  AND s.yearID NOT IN (1994, 1995)
  AND (s.salary = mm.max_salary OR s.salary = mm.min_salary)
ORDER BY s.yearID, key_player DESC, salary DESC;
'''
print(pd.read_sql_query(query7, conn))

      yearID   playerID nameFirst  nameLast    salary key_player
0       1985  birtsti01       Tim   Birtsas     60000        Min
1       1985  brownch02     Chris     Brown     60000        Min
2       1985  brownto05       Tom  Browning     60000        Min
3       1985  clibust02      Stew   Cliburn     60000        Min
4       1985  colemvi01     Vince   Coleman     60000        Min
...      ...        ...       ...       ...       ...        ...
1505    2016  walshco02     Colin     Walsh    507500        Min
1506    2016  whitety01     Tyler     White    507500        Min
1507    2016  wislema01      Matt    Wisler    507500        Min
1508    2016  wolteto01      Tony   Wolters    507500        Min
1509    2016  kershcl01   Clayton   Kershaw  33000000        Max

[1510 rows x 6 columns]


8. Average attendance for texas ranger games in each ballpark for years 1969-2023

In [23]:
query8 = '''
SELECT yearkey, AVG(attendance)
FROM homegames
WHERE teamkey IN ('TEX', 'WS2')
    AND yearkey BETWEEN 1969 AND 2023
    AND yearkey != 1981
    AND yearkey != 1994
    AND yearkey != 1995
    AND yearkey != 2020
GROUP BY yearkey
ORDER BY yearkey;
'''
print(pd.read_sql_query(query8, conn))

    yearkey  AVG(attendance)
0      1969         914227.0
1      1970         824750.0
2      1971         655156.0
3      1972         667287.0
4      1973         686162.0
5      1974        1193867.0
6      1975        1127663.0
7      1976        1161682.0
8      1977        1250721.0
9      1978        1447963.0
10     1979        1519672.0
11     1980        1198165.0
12     1982        1154432.0
13     1983        1362314.0
14     1984        1102430.0
15     1985        1112461.0
16     1986        1692021.0
17     1987        1763749.0
18     1988        1582084.0
19     1989        2042604.0
20     1990        2057917.0
21     1991        2297718.0
22     1992        2198231.0
23     1993        2252616.0
24     1996        2889040.0
25     1997        2923590.0
26     1998        2927409.0
27     1999        2774501.0
28     2000        2800075.0
29     2001        2831111.0
30     2002        2352397.0
31     2003        2094394.0
32     2004        2513685.0
33     2005   

9. The stadiums with the most attendence for the Texas Rangers per year. 

In [22]:
query9 = '''
SELECT homegames.yearkey, parks.parkname, MAX(homegames.attendance)
FROM homegames JOIN parks USING(parkkey)
WHERE homegames.teamkey IN ('TEX', 'WS2')
    AND homegames.yearkey BETWEEN 1969 AND 2023
    AND  homegames.yearkey != 1981
    AND  homegames.yearkey != 1994
    AND  homegames.yearkey != 1995
    AND  homegames.yearkey != 2020
GROUP BY  homegames.yearkey
ORDER BY  homegames.yearkey;
'''
print(pd.read_sql_query(query9, conn))

    yearkey                       parkname  MAX(homegames.attendance)
0      1969      Robert F. Kennedy Stadium                     914227
1      1970      Robert F. Kennedy Stadium                     824750
2      1971      Robert F. Kennedy Stadium                     655156
3      1972              Arlington Stadium                     667287
4      1973              Arlington Stadium                     686162
5      1974              Arlington Stadium                    1193867
6      1975              Arlington Stadium                    1127663
7      1976              Arlington Stadium                    1161682
8      1977              Arlington Stadium                    1250721
9      1978              Arlington Stadium                    1447963
10     1979              Arlington Stadium                    1519672
11     1980              Arlington Stadium                    1198165
12     1982              Arlington Stadium                    1154432
13     1983         

10. Exploring why average and maximum appear to be the same. 

In [24]:
query10 = '''
SELECT 
  yearkey, 
  parkkey,
  COUNT(*) AS games_played,
  AVG(attendance) AS avg_att,
  MAX(attendance) AS max_att
FROM homegames
GROUP BY yearkey, parkkey
ORDER BY games_played ASC;
'''
print(pd.read_sql_query(query10, conn))


      yearkey parkkey  games_played    avg_att  max_att
0        1871   BAL01             1     1000.0     1000
1        1871   BOS01             1    32600.0    32600
2        1871   CLE01             1     9500.0     9500
3        1871   FOR01             1     2350.0     2350
4        1871   PHI01             1    52000.0    52000
...       ...     ...           ...        ...      ...
3107     2017   NYC20             2  1255066.0  2460622
3108     2017   STP01             2   608551.0  1204109
3109     2019   MNT01             2    34735.0    35791
3110     1873   NYC01             3        0.0        0
3111     1871   NYC01             4    11787.5    42050

[3112 rows x 5 columns]


In [30]:
query11 = '''
SELECT parkkey, COUNT(*) AS games_played, AVG(attendance) AS avg_att, MAX(attendance) AS max_att
FROM homegames
WHERE teamkey IN ('TEX', 'WS2')
    AND yearkey BETWEEN 1969 AND 2023
    AND  yearkey != 1981
    AND  yearkey != 1994
    AND  yearkey != 1995
    AND  yearkey != 2020
GROUP BY parkkey
ORDER BY games_played ASC;
'''
print(pd.read_sql_query(query11, conn))


  parkkey  games_played       avg_att  max_att
0   ARL03             3  2.218221e+06  2533044
1   WAS10             3  7.980443e+05   914227
2   ARL01            21  1.470084e+06  2297718
3   ARL02            24  2.593138e+06  3460280


not many homegames played

11. Top highest attened rangers games

In [32]:
query12 = '''
SELECT *
FROM homegames h JOIN parks p USING(parkkey)
WHERE h.teamkey IN ('TEX', 'WS2')
ORDER BY h.attendance DESC
LIMIT 5;
'''
print(pd.read_sql_query(query12, conn))

   yearkey leaguekey teamkey parkkey   spanfirst    spanlast  games  openings  \
0     2012        AL     TEX   ARL02  2012-04-06  2012-09-30     81        81   
1     2013        AL     TEX   ARL02  2013-04-05  2013-09-30     82        82   
2     2011        AL     TEX   ARL02  2011-04-01  2011-09-25     81        81   
3     1998        AL     TEX   ARL02  1998-03-31  1998-09-20     81        81   
4     1997        AL     TEX   ARL02  1997-04-01  1997-09-21     81        80   

   attendance   ID                                 parkalias  \
0     3460280  260  The Ballpark in Arlington; Ameriquest Fl   
1     3178273  260  The Ballpark in Arlington; Ameriquest Fl   
2     2946949  260  The Ballpark in Arlington; Ameriquest Fl   
3     2927409  260  The Ballpark in Arlington; Ameriquest Fl   
4     2923590  260  The Ballpark in Arlington; Ameriquest Fl   

                        parkname       city state country  
0  Rangers Ballpark in Arlington  Arlington    TX      US  
1  Range

12. Year-by-year change in attendance

In [34]:
query13 = '''
WITH yearly_attendance AS (
  SELECT yearkey, AVG(attendance) AS avg_attendance
  FROM homegames
  WHERE teamkey IN ('TEX', 'WS2')
    AND yearkey NOT IN (1981, 1994, 1995, 2020)
  GROUP BY yearkey
)
SELECT 
  yearkey, avg_attendance, avg_attendance - LAG(avg_attendance) OVER (ORDER BY yearkey) AS change
FROM yearly_attendance;
'''
print(pd.read_sql_query(query13, conn))

    yearkey  avg_attendance    change
0      1961        597597.0       NaN
1      1962        729741.0  132144.0
2      1963        543631.0 -186110.0
3      1964        595271.0   51640.0
4      1965        562913.0  -32358.0
5      1966        589817.0   26904.0
6      1967        765788.0  175971.0
7      1968        541282.0 -224506.0
8      1969        914227.0  372945.0
9      1970        824750.0  -89477.0
10     1971        655156.0 -169594.0
11     1972        667287.0   12131.0
12     1973        686162.0   18875.0
13     1974       1193867.0  507705.0
14     1975       1127663.0  -66204.0
15     1976       1161682.0   34019.0
16     1977       1250721.0   89039.0
17     1978       1447963.0  197242.0
18     1979       1519672.0   71709.0
19     1980       1198165.0 -321507.0
20     1982       1154432.0  -43733.0
21     1983       1362314.0  207882.0
22     1984       1102430.0 -259884.0
23     1985       1112461.0   10031.0
24     1986       1692021.0  579560.0
25     1987 

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


13. Attendance and total wins

In [35]:
query14 = '''
WITH wins AS (
  SELECT yearID, teamID, W
  FROM teams
  WHERE teamID IN ('TEX', 'WS2')
),
attendance AS (
  SELECT yearkey, AVG(attendance) AS avg_att
  FROM homegames
  WHERE teamkey IN ('TEX', 'WS2')
    AND yearkey NOT IN (1981, 1994, 1995, 2020)
  GROUP BY yearkey
)
SELECT 
  a.yearkey,
  w.W AS wins,
  a.avg_att,
  ROUND(CAST(a.avg_att AS FLOAT) / w.W, 2) AS fans_per_win
FROM attendance a JOIN wins w ON a.yearkey = w.yearID
ORDER BY a.yearkey;
'''
print(pd.read_sql_query(query14, conn))

    yearkey  wins    avg_att  fans_per_win
0      1961    61   597597.0       9796.67
1      1962    60   729741.0      12162.35
2      1963    56   543631.0       9707.70
3      1964    62   595271.0       9601.15
4      1965    70   562913.0       8041.61
5      1966    71   589817.0       8307.28
6      1967    76   765788.0      10076.16
7      1968    65   541282.0       8327.42
8      1969    86   914227.0      10630.55
9      1970    70   824750.0      11782.14
10     1971    63   655156.0      10399.30
11     1972    54   667287.0      12357.17
12     1973    57   686162.0      12037.93
13     1974    84  1193867.0      14212.70
14     1975    79  1127663.0      14274.22
15     1976    76  1161682.0      15285.29
16     1977    94  1250721.0      13305.54
17     1978    87  1447963.0      16643.25
18     1979    83  1519672.0      18309.30
19     1980    76  1198165.0      15765.33
20     1982    64  1154432.0      18038.00
21     1983    77  1362314.0      17692.39
22     1984

Fans per win is the ratio of average attendance to number of wins. This could give insight on whether people are stil showing up regardes of the team wins.

# Acknowledgements
Arpan Gupta Data Scientist, IITian  (https://www.youtube.com/watch?app=desktop&v=psCNAXMSdYI) 
- SQL Queries in Kaggle Tutorial   (Thank you, very helpful :)   )